In [2]:
import cv2
import mediapipe as mp
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_pose = mp.solutions.pose

In [4]:
cap = cv2.VideoCapture("Joggen.MOV")
widest = -999999
lowest = 999999
safe_img_widest = None
with mp_pose.Pose(
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5) as pose:
  while cap.isOpened():
    success, image = cap.read()
    if not success:
      print("Ignoring empty camera frame.")
      print(widest)
      cv2.imshow("Widest IMG", safe_img_widest)
      # If loading a video, use 'break' instead of 'continue'.
      break

    # To improve performance, optionally mark the image as not writeable to
    # pass by reference.
    image.flags.writeable = False
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    results = pose.process(image)

    # Draw the pose annotation on the image.
    image.flags.writeable = True
    font = cv2.FONT_HERSHEY_SIMPLEX

    image_hight, image_width, _ = image.shape
    x_knee = results.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_KNEE].x * image_width
    y_knee = results.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_KNEE].y * image_hight
    x_ankle = results.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_ANKLE].x * image_width
    y_ankle = results.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_ANKLE].y * image_hight
    difference = x_knee-x_ankle
    

    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    mp_drawing.draw_landmarks(
        image,
        results.pose_landmarks,
        mp_pose.POSE_CONNECTIONS,
        landmark_drawing_spec=mp_drawing_styles.get_default_pose_landmarks_style())
    # Flip the image horizontally for a selfie-view display.
    image = cv2.putText(image, f'Linkes Knie zu Knöchel: {x_knee-x_ankle}', (50,50), font, 
                   1, (0,0,0), 4, cv2.LINE_AA)
    #image = cv2.putText(image, f'Rechtes Knie Y: {y_coodinate}', (50,100), font, 
    #               1, (0,0,0), 4, cv2.LINE_AA)
    cv2.imshow("Bildauschnitt", image)

    if difference > widest:
      widest = difference
      safe_img_widest = image.copy()
    if cv2.waitKey(5) & 0xFF == 27:
      break
cap.release()

Ignoring empty camera frame.
20.253907442092896
